# Mathematical Foundations of the FIPS-Certified PQC Algorithms

This notebook surveys the mathematics behind the three post-quantum cryptography (PQC) schemes standardized by NIST and published as FIPS 203 (ML-KEM / Kyber), FIPS 204 (ML-DSA / Dilithium), and FIPS 205 (SLH-DSA / SPHINCS+). It is designed as a reference companion to algorithm-focused notebooks: you can run the few lightweight Python snippets to build intuition for the underlying algebra, lattices, and hash trees used by the schemes.

## FIPS Landscape and Design Goals

- **Confidentiality**: ML-KEM (formerly Kyber) offers a Key Encapsulation Mechanism (KEM) based on the hardness of the Module-LWE problem over structured lattices.
- **Primary signature**: ML-DSA (formerly Dilithium) delivers a module-lattice signature with strong security margins and fast verification.
- **Fallback signature**: SLH-DSA (formerly SPHINCS+) is a stateless hash-based signature focusing on conservative, assumption-minimal security.
- **Common objectives**: quantum-resistant security, side-channel awareness, efficient implementations on constrained hardware, and a clear specification enabling FIPS certification.

## 1. Mathematical Preliminaries

The standardized schemes rely on overlapping primitives. This section reviews the shared mathematical structures.

### 1.1 Polynomial Rings Modulo $x^n + 1$
Let $R_q = \mathbb{Z}_q[x]/(x^n + 1)$ with $n$ a power of two and $q$ prime. Elements are polynomials
$$a(x) = a_0 + a_1 x + \dots + a_{n-1}x^{n-1} \pmod{x^n + 1, q}.$$
Negacyclic reduction ($x^n \equiv -1$) couples indices and enables structured lattices. Addition is coefficient-wise, multiplication wraps using the modulus polynomial and reduces modulo $q$.

### 1.2 $q$-ary Lattices and Modules
A lattice $\Lambda \subset \mathbb{R}^m$ is the integer span of linearly independent vectors. PQC schemes use **module lattices**, where basis vectors come from polynomial ring coefficients. For $k$-dimensional modules $R_q^k$, the lattice inherits the ring's negacyclic symmetry. Hardness assumptions generalize Learning With Errors (LWE) and Short Integer Solutions (SIS) to this module setting.

### 1.3 Module-LWE (MLWE) and Module-SIS (MSIS)
- **MLWE**: Given $(A, t = As + e)$ for random $A \in R_q^{k	imes k}$, secret $s$ drawn from a small distribution, and noise $e$, distinguish $t$ from uniform. ML-KEM directly encodes agreements as MLWE samples.
- **MSIS**: Find a short vector $z 
eq 0$ such that $Az = 0 mod q$. ML-DSA reduces unforgeability to the hardness of MSIS/MLWE, leveraging the Fiat-Shamir with aborts paradigm.

### 1.4 Discrete Gaussian and Centered Binomial Distributions
Secrets/noise use bounded, symmetric distributions approximating discrete Gaussians. For implementation, centered binomial distributions $	ext{CBD}_\eta$ sample differences of $\eta$ uniform bits, giving coefficients in $[-\eta, \eta]$ with tight bounds.

### 1.5 Hash-Based Combiners
SLH-DSA uses cryptographic hash functions (SHA-256 / SHAKE256) to build Winternitz One-Time Signatures (WOTS+), FORS (Forest of Random Subsets) signatures, and hypertree-based Merkle authentication. Security stems from the preimage and collision resistance of the underlying hashes.

In [ ]:
# Polynomial arithmetic helper to visualize R_q operations.
from typing import List

q = 3329  # Kyber modulus
n = 8     # toy degree (Kyber uses n=256)

poly_a = [5, 0, 1, 0, 0, 0, 0, 0]
poly_b = [2, 1, 0, 0, 0, 0, 0, 0]

def negacyclic_convolution(a: List[int], b: List[int]) -> List[int]:
    res = [0] * n
    for i, ai in enumerate(a):
        for j, bj in enumerate(b):
            k = i + j
            if k >= n:
                k -= n
                res[k] -= ai * bj
            else:
                res[k] += ai * bj
    return [c % q for c in res]

print("(5 + x) * (2 + x) mod (x^8 + 1,", q, ") =", negacyclic_convolution(poly_a, poly_b))

### 2. Number Theoretic Transform (NTT)
Polynomial multiplications dominate the cost of lattice-based crypto. The Number Theoretic Transform is a finite-field analogue of the FFT and maps coefficient representations into a point-wise domain.

- Choose a primitive $2n$-th root of unity $\psi$ modulo $q$ such that $\psi^n \equiv -1$.
- For polynomial $a$, compute $\widehat{a}_i = \sum_{j=0}^{n-1} a_j \omega^{ij}$ where $\omega = \psi^2$ is an $n$-th root.
- Negacyclic convolution becomes point-wise multiplication: $\widehat{c}_i = \widehat{a}_i \cdot \widehat{b}_i$.
- Apply the inverse transform (scaled by $n^{-1} mod q$) to get the coefficients back.

The NTT enables $O(n \log n)$ multiplication and underpins Kyber and Dilithium implementations.

In [ ]:
# Minimal NTT demo over a small prime modulus with n=8.
MOD = 17
N = 8
# primitive 16-th root of unity modulo 17 is 3 (since 3^8 = -1 mod 17)
PSI = 3
OMEGA = pow(PSI, 2, MOD)
INV_N = pow(N, -1, MOD)

def ntt(a):
    res = []
    for i in range(N):
        acc = 0
        for j, aj in enumerate(a):
            acc = (acc + aj * pow(OMEGA, i * j, MOD)) % MOD
        res.append(acc)
    return res

def intt(a):
    res = []
    for j in range(N):
        acc = 0
        for i, ai in enumerate(a):
            acc = (acc + ai * pow(OMEGA, -i * j, MOD)) % MOD
        res.append((acc * INV_N) % MOD)
    return res

sample = [1, 2, 3, 4, 0, 0, 0, 0]
ntt_sample = ntt(sample)
print("NTT(sample) =", ntt_sample)
print("Inverse NTT ->", intt(ntt_sample))

## 3. ML-KEM (Kyber) – FIPS 203

### 3.1 Core Structure
- Operates over $R_q$ with $n = 256$ and $q = 3329$.
- Module dimension $k \in \{2, 3, 4\}$ corresponds to ML-KEM-512/768/1024.
- Public matrix $A$ is expand-on-demand from a seed via SHAKE128, ensuring indifferentiability and compactness.
- Secrets $s, e$ sampled from $	ext{CBD}_\eta$ ($\eta=3$ or $2$ depending on level).

### 3.2 MLWE-Based Key Generation
1. Sample $s, e \leftarrow 	ext{CBD}$.
2. Compute $t = As + e$ in the NTT domain for speed.
3. Public key: $(t, 
ho)$; secret key: $(s, h(t), 	ext{pk})$ plus optional KDF material for CCA security.

### 3.3 Encapsulation / Decapsulation Math
- **Encap**: sample ephemeral secrets, compute $u = A^T r + e_1$, $v = t^T r + e_2 + m \cdot \lfloor q/2 
floor$ with message $m \in \{0,1\}^n$ encoded as noise-shifted polynomial. Derive shared key via KDF( hash(u, v) ).
- **Decap**: use $s$ to compute $m' = v - u^T s$, then re-encapsulate to mitigate chosen-ciphertext attacks (FIPS adopts IND-CCA via Fujisaki-Okamoto transform).

### 3.4 Parameter Highlights

| Variant | $k$ | $\eta_1$ | $\eta_2$ | Ciphertext bytes | Target security |
| --- | --- | --- | --- | --- | --- |
| ML-KEM-512 | 2 | 3 | 2 | 768 | ~128-bit quantum |
| ML-KEM-768 | 3 | 2 | 2 | 1088 | ~192-bit quantum |
| ML-KEM-1024 | 4 | 2 | 2 | 1568 | ~256-bit quantum |

### 3.5 Security Rationale
- Reduces to standard MLWE hardness with module rank $k$ and ring dimension $n$.
- Structured lattices allow speed but require careful parameter selection to avoid algebraic attacks; $q=3329$ prevents subfield attacks while keeping efficient NTT roots.
- Error bounds ensure decapsulation correctness with overwhelming probability, analyzed using tail bounds for $	ext{CBD}$.

In [ ]:
# Simulate an MLWE sample to visualize noise growth.
import random

def cbd_eta(eta, n=256):
    return [sum(random.getrandbits(1) for _ in range(eta)) -
            sum(random.getrandbits(1) for _ in range(eta)) for _ in range(n)]

def dot_mod(a, b, modulus=3329):
    return sum(x * y for x, y in zip(a, b)) % modulus

n = 16  # toy dimension for illustration
A_row = [random.randrange(3329) for _ in range(n)]
s = cbd_eta(3, n)
e = cbd_eta(3, 1)[0]
t = (dot_mod(A_row, s, 3329) + e) % 3329
print("Example MLWE sample: A_row[:5]=", A_row[:5])
print("s[:5]=", s[:5])
print("t=", t)

## 4. ML-DSA (Dilithium) – FIPS 204

### 4.1 Algebraic Backbone
- Works over the same ring $R_q$ (with $q = 8380417$, $n = 256$) but uses module rank $k, l$ tailored for signatures.
- Unforgeability reduces to simultaneous hardness of MLWE and MSIS by expressing signing as solving short vector equations masked by randomness.

### 4.2 Fiat–Shamir with Aborts
1. Expand matrix $A \in R_q^{k 	imes l}$ via SHAKE128 using public seed $
ho$.
2. Sample secret vectors $(s_1, s_2)$ with small coefficients.
3. To sign message $\mu$:
   - Sample ephemeral $y$ from a bounded distribution.
   - Compute $w = A y$ and split into high/low bits: $w = w_1 + w_0$ with $w_1$ exposed.
   - Challenge $c = H(\mu, w_1)$ (modeled as random oracle).
   - Response $z = y + c s_1$ must remain within bounds; otherwise abort and retry.
   - Pack hint bits $h$ to reconstruct $w_1$ during verification.
4. Verification recomputes $w'$ and ensures consistency plus norm bounds on $z$.

### 4.3 Parameter Table

| Variant | $(k, l)$ | $\eta$ | $\gamma_1$ | $\gamma_2$ | Signature bytes | Target |
| --- | --- | --- | --- | --- | --- | --- |
| ML-DSA-44 | (4, 4) | $(3, 2)$ | $2^{17}$ | $88$ | 2044 | ~128-bit |
| ML-DSA-65 | (6, 5) | $(4, 4)$ | $2^{19}$ | $32$ | 3556 | ~192-bit |
| ML-DSA-87 | (8, 7) | $(5, 5)$ | $2^{19}$ | $32$ | 4907 | ~256-bit |

### 4.4 Mathematical Intuition
- **Rejection sampling** enforces a "Gaussian-like" distribution for $z$ even after adding $c s_1$, preventing leakage of $s_1$.
- **Hints and rounding**: splitting $w$ into high/low bits approximates the coset of a lattice; verifying ensures the challenger reconstructs the same coset.
- **Security reductions** map a forgery to a short vector in the module lattice, contradicting MSIS hardness.

### 4.5 Side-Channel and Implementation Notes
- Deterministic expansions of $A$ and PRF-based masking protect against fault and timing attacks.
- Sparse challenge polynomials $c$ (with 60 non-zero coefficients in ML-DSA-87) reduce computation while maintaining unpredictability.

In [ ]:
# Illustrate rejection sampling by counting how often z exceeds a bound.
import random
import statistics

def cbd_eta(eta, n=256):
    """Centered Binomial Distribution sampling function."""
    return [sum(random.getrandbits(1) for _ in range(eta)) -
            sum(random.getrandbits(1) for _ in range(eta)) for _ in range(n)]

BOUND = 60
trials = 1000
rejections = 0
lengths = []
for _ in range(trials):
    y = cbd_eta(4, 64)
    c = [0]*64
    for i in random.sample(range(64), 5):
        c[i] = random.choice([-1, 1])
    s1 = cbd_eta(4, 64)
    z = [y_i + c_i * s_i for y_i, c_i, s_i in zip(y, c, s1)]
    norm_inf = max(abs(val) for val in z)
    lengths.append(norm_inf)
    if norm_inf >= BOUND:
        rejections += 1
print(f"Rejection probability ≈ {rejections/trials:.2%} with toy parameters")
print("Average infinity-norm:", statistics.mean(lengths))

NameError: name 'cbd_eta' is not defined

## 5. SLH-DSA (SPHINCS+) – FIPS 205

### 5.1 Hash-Based Security Model
- Security derived solely from the hash function's preimage and collision resistance.
- Stateless design avoids reliance on consistent private randomness during signing (a common pitfall in hash-based schemes).

### 5.2 Building Blocks
- **WOTS+ (Winternitz One-Time Signature)**: signs short digests using iterative hash chains. Parameter $w$ balances signature size versus time.
- **FORS (Forest of Random Subsets)**: few-time signature authenticating compressed messages by revealing paths in small Merkle trees.
- **Hypertree**: $d$ layered Merkle trees glue many WOTS+/FORS leaves, enabling up to $2^{64}$ signatures per key pair without state.

### 5.3 Signing Flow (High Level)
1. Derive secret seeds $(SK_{seed}, SK_{prf})$ and public seed $PK_{seed}$.
2. To sign message $M$:
   - Compute message digest and tree index using tweakable hash $H( PK, M )$.
   - Generate FORS leaves/paths for the digest bits.
   - For each layer of the hypertree, produce a WOTS+ signature for the node below and authenticate it with a Merkle path.
3. Public key: $(PK_{seed}, root)$ where `root` is the top-level Merkle root.

### 5.4 Parameter Highlights (SHAKE256 simple variants)

| Variant | n (bytes) | $h$ | $d$ | WOTS $w$ | Signature bytes | Target |
| --- | --- | --- | --- | --- | --- | --- |
| SLH-DSA-SHAKE-128s | 16 | 63 | 7 | 16 | 7856 | ~128-bit |
| SLH-DSA-SHAKE-192s | 24 | 63 | 7 | 16 | 16224 | ~192-bit |
| SLH-DSA-SHAKE-256s | 32 | 66 | 7 | 16 | 29792 | ~256-bit |

Variants with larger `f` suffix trade smaller signatures for slower verification by increasing $w$ and reducing $h$.

### 5.5 Security Intuition
- Winternitz chains behave as one-way functions; forging requires inverting hash steps.
- FORS reduces long messages to multiple small trees, distributing security so that breaking the scheme needs many hash inversions.
- Merkle trees bind everything together: the root serves as the public key, and each authentication path proves inclusion.

In [ ]:
# Toy Merkle tree to visualize SLH-DSA authentication.
import hashlib

def H(data: bytes) -> bytes:
    return hashlib.sha256(data).digest()

def merkle_tree(leaves):
    nodes = [leaf for leaf in leaves]
    tree = [nodes]
    while len(nodes) > 1:
        nxt = []
        for i in range(0, len(nodes), 2):
            pair = nodes[i:i+2]
            if len(pair) == 1:
                pair.append(pair[0])
            nxt.append(H(pair[0] + pair[1]))
        nodes = nxt
        tree.append(nodes)
    return tree

messages = [f"Leaf {i}".encode() for i in range(4)]
leaves = [H(m) for m in messages]
tree = merkle_tree(leaves)
root = tree[-1][0]
print("Merkle root:", root.hex())
print("Authentication path for Leaf 2:")
node_idx = 2
for level, nodes in enumerate(tree[:-1]):
    sibling = node_idx ^ 1
    print(f" level {level}: sibling hash = {nodes[sibling].hex()}")
    node_idx //= 2

## 6. Comparative Insights

- **Algebraic versus combinatorial hardness**: ML-KEM and ML-DSA share lattice/module assumptions and efficient polynomial arithmetic, while SLH-DSA relies solely on hash-function security.
- **Performance**: ML-KEM offers the smallest ciphertexts and best throughput; ML-DSA balances signing cost with verification speed; SLH-DSA produces larger signatures but provides a conservative safety net.
- **Implementation focus**: hardware acceleration of the NTT and constant-time rejection sampling are crucial for ML-KEM/ML-DSA. SPHINCS+ favors parallel hash pipelines and memory bandwidth.
- **Parameter selection**: each FIPS document details strict bounds on decryption failure (ML-KEM), signature rejection probability (ML-DSA), and multi-target security for the hash tree (SLH-DSA). Designers must track these bounds when porting or optimizing implementations.

## 7. Further Reading

- **FIPS 203** – *Module-Lattice-Based Key-Encapsulation Mechanism Standard*
- **FIPS 204** – *Module-Lattice-Based Digital Signature Standard*
- **FIPS 205** – *Stateless Hash-Based Digital Signature Standard*
- NIST PQC project site: https://csrc.nist.gov/projects/post-quantum-cryptography
- Kyber, Dilithium, and SPHINCS+ specification papers (Round-3 submissions) for deeper security proofs.
- Oded Regev, "On lattices, learning with errors, random linear codes, and cryptography" (STOC 2005) – foundational LWE hardness result.
- Bernstein, et al., "Post-Quantum Cryptography" (Springer, 2009) – survey covering both lattice and hash-based approaches.